In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install emot --upgrade
!pip install emoji
!pip install demoji
!pip install contractions

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from sklearn.naive_bayes import MultinomialNB
from textblob import TextBlob
import emot
import nltk.data
import numpy as np
import pandas as pd
import re
import time
import string
import emoji
import demoji
import contractions
import nltk
nltk.download('omw-1.4')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option("max_colwidth" ,220)
wn = nltk.WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
demoji.download_codes()
def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

<ipython-input-29-e00aa54160fe>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
demoji.download_codes()
def emo(text):
  try:
   temp=emoji.demojize(str(text),delimiters=(" "," "))
   temp=temp.replace("_","  ")
   return temp
  except IndexError:
        return ""

<ipython-input-30-82a5fb4117d1>:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [ ]:
punct="!#$%&\'()*+,-.’’/:;<=>?@[\\]^_`{|}~’“‘"

In [ ]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [ ]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', str(text))

In [ ]:
def clean_text(text):
    text = re.sub('@[a-zA-Z0-9]*', '', str(text))#to remove @ and its following word
    text = contractions.fix(text, slang=True)
    text = "".join([word.lower() for word in text if word not in string.punctuation]) #to remove punctuation
    text="".join([word.lower() for word in text if word not in punct])
    text = "".join([word for word in text if not word.isdigit()])#to remove digit
    text = " ".join(word for word in text.split() if word not in stopwords)
    return text

In [ ]:
import pandas as pd


In [ ]:
bodo_train=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/train_BO_AH_HASOC2023.csv')
bodo_test=pd.read_csv('/content/drive/MyDrive/hasoc/task4/bodo/test_BO_AH_HASOC2023.csv')

In [ ]:
len(bodo_train)

1679

In [ ]:
len(bodo_test)

420

In [ ]:
bodo_train['task_1'].value_counts()

HOF    998
NOT    681
Name: task_1, dtype: int64

In [ ]:
bodo_train.shape,bodo_test.shape

((1679, 3), (420, 2))

In [ ]:
bodo_train['clean_text'] = bodo_train['text'].apply(lambda x:emo(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:remove_urls(x))
bodo_train['clean_text'] = bodo_train['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_train['clean_text']

0                                                                                                 गोदाव खामानि मावओ बोला नो सानसे देरहा थारगोन
1                                                                                                                      निखावरि सुबुंफोरा सिखाव
2                                                                                                   मा बिमा खर परिबर्थननि खोथा फैखो बेयाव मोसौ
3                                                                                                      थोद जामबा सैमा साला मा मिसेस जाखो बेलाय
4                                                      माखौ बकिबाय थादों नों बोरमा फानथा दम दंब्ला खामानि मावना दिन्थि बिनि उनाव बखि सैमा साला
                                                                         ...                                                                  
1674                                                                   नोंलाय जामबा नोंबो सासे सनमान गैयै मानसिसो गिदिरसिन मानसिखौ मुं मुखग्रा

In [ ]:
bodo_test['clean_text'] = bodo_test['text'].apply(lambda x:emo(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:remove_urls(x))
bodo_test['clean_text'] = bodo_test['clean_text'].apply(lambda x:clean_text(x))

In [ ]:
bodo_test['clean_text']

0                                                                                                      bpf बानाय लांनाय लामाया बोसोरानो जोरासै
1                                                                                  बै समाव माबेयाव हाबसोनानै दंमोन नोंलाय दाना बुजिना जाथारबाय
2      बे थांखिखौ मिनिग्रापोरा हारिखौ लेवारपोरबायदि थै सोबग्रासो। राजनिथिक गादिनि गोदै गोबाब स्वाद मोनजानानै गावनि हारिखौ निन्दिग्रा मिनिग्रा।
3                                                                                                         मोसौ खुगायाव एमफौ नांबाय नोंनाव सैमा
4                                                                     आव btc गोरोबथा जादों बेनि थाखाय बो थोजासे सम होदों नाथाय राज्योराजाफोरखौ
                                                                        ...                                                                   
415                                                                                                                      आं आनो खाजा होआखै मोन

In [ ]:
import regex
def custom_analyzer(text):
    words = regex.findall(r'\w{2,}', text) #extract words of at least 2 letters
#     for w in words:
    for w in words:
      yield w

In [ ]:
X_train=bodo_train['clean_text']
X_test=bodo_test['clean_text']
y_train=bodo_train['task_1']

In [ ]:
#feature extraction
# word level tf-idf
tfidf_vect1 = TfidfVectorizer(analyzer=custom_analyzer)
tfidf_vect1.fit(X_train)
xtrain_tfidf1 =  tfidf_vect1.transform(X_train)
xtest_tfidf1=  tfidf_vect1.transform(X_test)

In [ ]:
xtrain_tfidf1 .shape,xtest_tfidf1.shape

((1679, 4741), (420, 4741))

In [ ]:
from sklearn.svm import SVC
svm = SVC()

svm .fit(xtrain_tfidf1, y_train)

y_pred = svm.predict(xtest_tfidf1)


In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred, columns=['y_pred'])
Submisssion_bodo = pd.DataFrame()
Submisssion_bodo['S. No.'] = bodo_test['S. No.']
Submisssion_bodo['task_1'] = y_pred_df
Submisssion_bodo.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run1.csv',index = None)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC

clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)

eclf = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2),
('SVC', clf3)], voting='hard')
eclf = eclf.fit(xtrain_tfidf1, y_train)
y_pred_df = eclf.predict(xtest_tfidf1)
print(y_pred_df)

['NOT' 'NOT' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'NOT'
 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'NOT' 'HOF'
 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'NOT'
 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'HOF' 'NOT'
 'HOF' 'HOF' 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT'
 'NOT' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'NOT'
 'NOT' 'HOF' 'HOF' 'NOT' 'HOF' 'NOT' 'NOT' 'HOF' 'HOF' 'HOF' 'NOT' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF'
 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'HOF' 'H

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df= pd.DataFrame(data=y_pred, columns=['y_pred'])
Submisssion_bodo = pd.DataFrame()
Submisssion_bodo['S. No.'] = bodo_test['S. No.']
Submisssion_bodo['task_1'] = y_pred_df
Submisssion_bodo.to_csv('/content/drive/MyDrive/MUCS_test_bodo_run2.csv',index = None)